In [1]:
import pandas as pd
import collections
import math
import os

import cv2
import matplotlib.pyplot as plt
import cv2
import sys
import numpy as np
import math
import time
from PIL import Image 
import shutil

from pandas import read_csv  
import configparser

config = configparser.ConfigParser()
config.read('XXX-config.txt')

source_folder = config.get('Configuration', 'source_folder')
time_step = int(config.get('Configuration', 'time_step'))
data_folder = config.get('Configuration', 'data_folder')
ini_date = config.get('Configuration', 'ini_date')
end_date = config.get('Configuration', 'end_date')

rooms=config.get('Configuration', 'rooms')
rooms = [(item.strip()) for item in rooms.split(',')]

users=config.get('Configuration', 'users')
users = [(item.strip()) for item in users.split(',')]

anchors=config.get('Configuration', 'anchors')
anchors = [(item.strip()) for item in anchors.split(',')]

w = int(config.get('Configuration', 'w'))
h = int(config.get('Configuration', 'h'))

user_color={}
for user in users:
    user_color[user]=config.get('Colors', user)
print(user_color)


{'f02e': 'blue', '867f': 'red', 'ed9c': 'green'}


In [2]:
import numpy as np
from pandas import read_csv    
import pandas as pd
import time

from datetime import datetime

off_zone=60*60*2

def day_time(ti):
    return int((int)((ti+off_zone)/(60*60*24)))


def time2str(tt):
    return datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S")


t0 = (int)(datetime.strptime(ini_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (ini_date,"Init date is", t0, "day:",day_time(t0), time2str(t0))

tN = (int)(datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (end_date,"End date is", tN, "day:",day_time(tN), time2str(tN))


def day_time0(ti):
    return day_time(ti)-day_time(t0)+1

def relT(ti):
    return (int)((ti-t0)/time_step)

day=list(range(day_time(t0),day_time(tN)+1))[0]
day=19798
print(day)

1980-01-01 13:00:02 Init date is 315576002 day: 3652 1980-01-01 13:00:02
1980-01-01 14:51:20 End date is 315582680 day: 3652 1980-01-01 14:51:20
19798


In [3]:
i0=0
ts=list(range(t0,tN,1))
#print(i0,ts)

loc_sensor={}

def getLoc(room,s,oH,oW):
    print("S:",s)
    xy=config.get(room, s)
    xy = [(item.strip()) for item in xy.split(',')]
    xy = [list(map(int, item.split('x'))) for item in xy]
    return [xy[0][0]+oH,xy[0][1]+oW]

offW= {}
offH={}

for room in rooms:
    print(room)

    offW[room] = int(config.get(room, 'offW'))
    offH[room] = int(config.get(room, 'offH')) 
    
for room in rooms:
    for anchor in anchors:
        try:
            loc_sensor[anchor]=getLoc(room,anchor,offH[room],offW[room])
        except:
            print("None sensor",room,anchor)
print(loc_sensor)


livingroom
kitchen
bathroom
S: 8171
S: eb38
S: 1fa0
None sensor livingroom 1fa0
S: a304
None sensor livingroom a304
S: c492
None sensor livingroom c492
S: 8171
None sensor kitchen 8171
S: eb38
None sensor kitchen eb38
S: 1fa0
S: a304
S: c492
None sensor kitchen c492
S: 8171
None sensor bathroom 8171
S: eb38
None sensor bathroom eb38
S: 1fa0
None sensor bathroom 1fa0
S: a304
None sensor bathroom a304
S: c492
{'8171': [0, 5], 'eb38': [4, 2], '1fa0': [9, 4], 'a304': [14, 1], 'c492': [10, 8]}


In [4]:
from datetime import datetime
import os
from tkinter import *
from PIL import ImageTk, Image
import time


d0=40


def getXY(i,j):
    return j*d0+5,i*d0+5

def getFPImg(tt):
    return np.array(Image.open(data_folder+"/DAY_"+str(day)+"/FP."+str(tt)+".png"))/255.0

#def getSceneImg(scene):
#    return np.array(Image.open(data_folder+"/"+room+"/DAY_"+str(day)+"/FP.mean."+str(scene)+".png"))/255.0

def getGTImg(tt,user):
    return np.array(Image.open(data_folder+"/DAY_"+str(day)+"/GT."+user+"."+str(tt)+".png"))/255.0    

def getUserAnchor(user,anchor,tt):
    #print(data_folder+"/DAY_"+str(day)+"/"+user+"."+anchor+".tsv")
    df_total=read_csv(data_folder+"/DAY_"+str(day)+"/"+user+"."+anchor+".tsv", sep="\t",parse_dates=True, header=None, names=["time","d", "d2"])
    #print(df_total)
    df_intervalo = df_total[(df_total['time'] >= (tt-1)) & (df_total['time'] <= (tt)) & (df_total['d'] != -1)]
    #print("df_in#tervalo",df_intervalo)
    if(df_intervalo.size>0):
        d=np.mean(df_intervalo['d'])
        #print("d",d)
        #print(d,1-((d - rssi_min) / (rssi_max - rssi_min)))
        #if(d!=-1):
        #    return 1-((d - rssi_min) / (rssi_max - rssi_min))
        if(d<10):
            return (1-d/10.0)*(1-d/10.0)
        else:
            return 0.0
    return -1 
    
    

def getTime():
    return str(int(time.time()))+"\t"+datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def key_file_time():
    return str((int)(time.time()/(60*60*24)))

def key_file_time_now():
    return str((int)(time.time()))

def create_circle(x, y, r, canvasName): #center coordinates, radius
    x0 = x - r
    y0 = y - r
    x1 = x + r
    y1 = y + r
    return canvasName.create_oval(x0, y0, x1, y1, fill="red")

def motion(event):
    x, y = event.x, event.y
    print("Current Position = ", (x, y))
    create_circle(x, y, 25, canvas)

def display_text(text, x, y, tag="myText"):
    """
    Display text on the canvas at the specified (x, y) location.
    If text with the same tag exists, it updates the text instead of creating a new one.
    """
    existing_text = canvas.find_withtag(tag)
    if existing_text:
        canvas.itemconfigure(existing_text, text=text)
        canvas.coords(existing_text, x, y)
    else:
        canvas.create_text(x, y, text=text, fill="black", font=('Helvetica', '12'), tags=(tag,))    
    
def pintar_datos():
    global i0
    
    if(i0>len(ts)):
        return
    
    tt=ts[i0]
    print(tt)

    # Obtener las dimensiones del lienzo
    

    ancho = canvas.winfo_reqwidth()
    alto = canvas.winfo_reqheight()
    



    for user in users:
        for anchor in anchors:
            canvas.delete("point"+user+"x"+anchor)
            
    for i in range(h):  
        for j in range(w): 
            canvas.delete("red"+str(i)+"x"+str(j))
            
    for user in users:
        for i in range(h):  
            for j in range(w): 
                canvas.delete("gt"+user+"x"+str(i)+"x"+str(j))

    fpimg=getFPImg(tt)

    
    #print("fpimg",fpimg)

        

    for user in users:
        gtimg=getGTImg(tt,user)
        for i in range(h):  
            for j in range(w): 
                x,y=getXY(i,j)
                d=d0*gtimg[i][j]
                
                #print("user:",user,i,j,gtimg[i][j],x,y)
                canvas.create_oval(x - d, y - d, x + d, y + d, outline=user_color[user], fill=user_color[user], tags="gt"+user+"x"+str(i)+"x"+str(j))
    
    for i in range(h):  
        for j in range(w): 
            x,y=getXY(i,j)
            #d=d0*(fpimg[i][j]-img00[i][j])
            d=d0*(fpimg[i][j])
            canvas.create_oval(x - d, y - d, x + d, y + d, outline="grey", fill="grey", tags="red"+str(i)+"x"+str(j))

    for user in users:
        for anchor in anchors:
            d=getUserAnchor(user,anchor,tt)*d0
            if(d<0):
                d=2
            x,y=loc_sensor[anchor]
            x,y=getXY(x,y)
            #print(user,anchor,d)
            canvas.create_oval(x - d, y - d, x + d, y + d, outline=user_color[user], tags="point"+user+"x"+anchor)
    display_text("#"+time2str(tt), 50, 5, tag="greeting")
    i0=i0+1
    canvas.after(125, pintar_datos)    

win = Tk()

img = ImageTk.PhotoImage(Image.open("mapa-RADAR-sim.png").resize((w*d0, h*d0)))
canvas = Canvas(win, width=w*d0,height=h*d0)
canvas.create_image(0, 0,anchor=NW, image=img)
canvas.bind('<Button 1>', motion)

canvas.pack()

pintar_datos()


win.mainloop()

315576002
315576003
315576004
315576005
315576006
315576007
315576008
315576009
315576010
315576011
315576012
315576013
315576014
315576015
315576016
315576017
315576018
315576019
315576020
315576021
315576022
315576023
315576024
315576025
315576026
315576027
315576028
315576029
315576030
315576031
315576032
315576033
315576034
315576035
315576036
315576037
315576038
315576039
315576040
315576041
315576042
315576043
315576044
315576045
315576046
315576047
315576048
